# Лабораторная работа №2
## Выполнил студент группы БСТ1904 Костин Антон Алексеевич
### Задание №1

Реализовать методы поиска в соответствии с заданием. Организовать генерацию
начального набора случайных данных. Для всех вариантов добавить реализацию
добавления, поиска и удаления элементов. Оценить время работы каждого алгоритма
поиска и сравнить его со временем работы стандартной функции поиска, используемой в
выбранном языке программирования. 

|Бинарный поиск|Бинарное дерево|Фибоначчиев|Интерполяционный|
|------|------|------|------|

### Бинарный поиск

In [2]:
class BinarySearch:
    def __init__(self):
        self.data = []

    def search(self, value):
        left = 0
        right = len(self.data)

        while left < right:
            middle = (left + right) // 2
            if self.data[middle][0] < value:
                left = middle + 1
            else:
                right = middle
        
        return left

    def __setitem__(self, key, value):
        index = self.search(key)
        if index < len(self.data) and self.data[index][0] == key:
            self.data[index] = (key, value)
        else:
            self.data.insert(index, (key, value))

    def __delitem__(self, key):
        index = self.search(key)
        self.data.pop(index)

    def __getitem__(self, key):
        index = self.search(key)
        foundKey, val = self.data[index]
        if foundKey == key:
            return val
        else:
            return None

### Бинарное дерево
Значения меньше корневого находятся в левом поддереве, а большие - в правом

In [3]:
class Node:
    def __init__(self, key):
        self.key = key
        self.left = None
        self.right = None
    
    def inorder(root):
        if root is not None:
            inorder(root.left)
            print root.key,
            inorder(root.right)

    def insert(node, key):
        if node is None:
            return Node(key)

        if key < node.key:
            node.left = insert(node.left, key)
        else:
            node.right = insert(node.right, key)
            
        return node
    
    def minValueNode(node):
        current = node
        while(current.left is not None):
            current = current.left
        return current
    
    def deleteNode(root, key):
        if root is None:
            return root
        
        if key < root.key:
            root.left = deleteNode(root.left, key)
        elif(key > root.key):
            root.right = deleteNode(root.right, key)
        else:
            if root.left is None:
                temp = root.right
                root = None
                return temp
            elif root.right is None:
                temp = root.left
                root = None
                return temp
            temp = minValueNode(root.right) 
            root.key = temp.key
            root.right = deleteNode(root.right, temp.key)
        return root

### Фибоначчиев поиск
В этом поиске анализируются элементы, находящиеся в позициях, равных числам Фибоначчи.  
Числа Фибоначчи получаются по следующему правилу: последующее число равно сумме двух предыдущих чисел.  
##### [0, 1, 2, 3, 5, 8, 13, 21, 34 ....]

In [4]:
fib_c = [0, 1]
def fibonnaci(n):
    if len(fib_c) - 1 < n:
        fib_c.append(fibonnaci(n - 1) + fibonnaci(n - 2))
    return fib_c[n]

class FibonacciSearch(BinarySearch):
    
    def search(self, key):  
        m = 0 
        while fibonnaci(m) < len(self.data): 
            m += 1
        offset = 0
        while fibonnaci(m) > 1:
            i = min(offset + fibonnaci(m - 1), len(self.data) - 1)
            if key > self.data[i][0]:
                offset = i
                m -= 1
            elif key < self.data[i][0]:
                m -=2
            else:
                return i
        if len(self.data) and self.data[offset][0] < key:
            return offset + 1
        return 0

### Интерполяционный поиск

In [5]:
def nearMid(arr, low, high, val):
    return low + (high - low) * (val - arr[low][0]) // (arr[high][0] - arr[low][0])
    
class InterpolateSearch(BinarySearch):
    def searchINT(self, value):
        size_of_list = len(self.data) - 1

        first = 0
        last = size_of_list

        while first <= last:
            middle = nearMid(self.data, first, last, value)

            if middle > last or middle < first:
                return None

            if self.data[middle][0] == value:
                return middle

            if value > self.data[middle][0]:
                first = middle + 1
            else:
                last = middle - 1

        if first > last:
            return None

### Задание 2
|Простое рехэширование|Рехэширование с помощью псевдослучайных чисел|Метод цепочек|
|------|------|------|

### Простое рехэширование


In [6]:
class HashMap:
    def __init__(self):
        self.size = 0
        self.data = []
        self._resize()
    
    def _hash(self, key, i):
        return (hash(key) + i) % len(self.data)
        
    def _find(self, key):
        i = 0;
        index = self._hash(key, i);
        while self.data[index] is not None and self.data[index][0] != key:
            i += 1
            index = self._hash(key, i);
        return index;
    
    def _resize(self):
        temp = self.data
        self.data = [None] * (2*len(self.data) + 1)
        for item in temp:
            if item is not None:
                self.data[self._find(item[0])] = item
    
    def __setitem__(self, key, value):
        if self.size + 1 > len(self.data) // 2:
            self._resize()
        index = self._find(key)
        if self.data[index] is None:  
            self.size += 1
        self.data[index] = (key, value)
    
    def __getitem__(self, key):
        index = self._find(key)
        if self.data[index] is not None:
            return self.data[index][1]
        raise KeyError()

### Рехеширование с помощью всевдослучайных чисел

In [7]:
class RandomHashMap(HashMap):
    _rand_c = [5323]
    
    def _rand(self, i):
        if len(self._rand_c) - 1 < i:
            self._rand_c.append(self._rand(i - 1))
        return (123456789 * self._rand_c[i] + 987654321) % 65546
        
    def _hash(self, key, i):
        return (hash(key) + self._rand(i)) % len(self.data)

### Метод цепочек

In [8]:
class ChainMap:
    def __init__(self):
        self.size = 0
        self.data = []
        self._resize()
    
    def _hash(self, key):
        return hash(key) % len(self.data)
    
    def _insert(self, index, item):
        if self.data[index] is None:
            self.data[index] = [item]
            return True
        else:
            for i, item_ in enumerate(self.data[index]):
                if item_[0] == item[0]:
                    self.data[index][i] = item
                    return False
            self.data[index].append(item)
            return True
    
    def _resize(self):
        temp = self.data
        self.data = [None] * (2*len(self.data) + 1)
        for bucket in temp:
            if bucket is not None:
                for key, value in bucket:
                    self._insert(self._hash(key), (key, value))
    
    def __setitem__(self, key, value):
        if self.size + 1 > len(self.data) // 1.5:
            self._resize()
        if self._insert(self._hash(key), (key, value)):  
            self.size += 1
    
    def __getitem__(self, key):
        index = self._hash(key)
        if self.data[index] is not None:
            for key_, value in self.data[index]:
                if key_ == key:
                    return value
        raise KeyError()

### Сравнение алгоритмов


In [24]:
import time

binary = BinarySearch()
fibon = FibonacciSearch()
interpolate = InterpolateSearch()
arr = []
for x in range(100000):
    binary[x] = x + 43
    fibon[x] = x + 43
    interpolate[x] = x + 43
    arr.append(x + 43)


binary_start = time.time()
print(binary[872])
binary_end = time.time()
print("Бинарный поиск: ", (binary_end - binary_start))

fib_start = time.time()
print(fibon[872])
fib_end = time.time()  
print("Поиск Фибонначи: ", (fib_end - fib_start))

int_start = time.time()
print(interpolate[interpolate.searchINT(872)])
int_end = time.time()  
print("Интерполяционный поиск: ", (int_end - int_start))

search_start = time.time()
print(arr[872])
search_end = time.time()
print("Встроенный поиск: ", (search_end - search_start))

915
Бинарный поиск:  0.000997781753540039
915
Поиск Фибонначи:  0.0
915
Интерполяционный поиск:  0.0
915
Встроенный поиск:  0.0


### Задание 3
Расставить на стандартной 64-клеточной шахматной доске 8 ферзей так, чтобы ни один из них не находился под боем другого». Подразумевается, что ферзь бьёт все клетки, расположенные по вертикалям, горизонталям и обеим диагоналям Написать программу, которая находит хотя бы один способ решения задач.

##### ИЛИ

Заполнить матрицу размером 8 х 8 нулями и единицами таким образом, чтобы сумма всех элементов матрицы была равна 8, при этом сумма элементов ни в одном столбце, строке или диагональном ряде матрицы не превышала единицы
